In [2]:
import torch
from ultralytics import YOLO
import time
import yaml

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
# 데이터셋 경로 설정 (yolo는 dataset 경로를 images와 labels로 해주어야 인식이 가능)
data_root = 'D:\\Falldown\\Dataset\\Resized_Dataset'
train_root = f'{data_root}\\Train\\images'
val_root = f'{data_root}\\Val\\images'
test_root = f'{data_root}\\Test\\images'

# 클래스 설정
class_names = {0 : 'Human'} # 팀원 yaml에서는 0을 비낙상, 1을 낙상으로 클래스 설정함
num_classes = len(class_names)

# yaml 설정
yaml_info = {
    'path' : r'D:\Falldown', # YOLOv8 기본 경로
    'names': class_names,
    'nc': num_classes,
    'train': train_root,
    'val': val_root,
    'test': test_root
}

# YAML 파일 저장 경로
yaml_file_path = 'D:\\Falldown\\yaml_info_yolov8s.yaml'

# YAML 파일 생성
with open(yaml_file_path, 'w') as f:
    yaml.dump(yaml_info, f)

print(f'yaml 파일이 생성되었습니다: {yaml_file_path}')

yaml 파일이 생성되었습니다: D:\Falldown\yaml_info_yolov8s.yaml


In [20]:
model = YOLO('yolov8s.pt')

In [21]:
try:
    # 학습 시작 시간 기록
    start_time = time.time()

    # 학습 실행
    result = model.train(
        data='D:\\Falldown\\yaml_info_yolov8s.yaml',
        epochs=50,
        batch=16,
        imgsz=640,
        device=device,
        workers=12,
        amp=True,
        patience=30,
        name='human_fall_s'
    )

    # 학습 종료 시간 기록
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"실행 시간: {execution_time:.4f} 초")  # 약 20시간 소요

except KeyboardInterrupt:
    print("KeyboardInterrupt: 사용자가 학습을 중단하였습니다.")

New https://pypi.org/project/ultralytics/8.3.38 available  Update with 'pip install -U ultralytics'
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=D:\Falldown\yaml_info_yolov8s.yaml, epochs=50, time=None, patience=30, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=12, project=None, name=human_fall_s2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_label

train: Scanning D:\Falldown\Dataset\Resized_Dataset\Train\labels... 76130 images, 0 backgrounds, 0 corrupt: 100%|██████████| 76130/76130 [00:30<00:00, 2479.50it/s]


train: New cache created: D:\Falldown\Dataset\Resized_Dataset\Train\labels.cache


val: Scanning D:\Falldown\Dataset\Resized_Dataset\Val\labels... 9440 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9440/9440 [00:04<00:00, 1975.29it/s]


val: New cache created: D:\Falldown\Dataset\Resized_Dataset\Val\labels.cache
Plotting labels to runs\detect\human_fall_s2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 12 dataloader workers
Logging results to runs\detect\human_fall_s2
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      4.09G     0.9372      5.722      1.135         21        640:   1%|▏         | 62/4759 [00:14<18:52,  4.15it/s] 

KeyboardInterrupt: 사용자가 학습을 중단하였습니다.


In [5]:
# 검증 시작 시간 기록
start_time = time.time()

# YOLO 모델 로드 및 검증 실행
model = YOLO('D:\\Falldown\\code-git\\runs\\detect\\human_fall_s30\\weights\\best.pt')
val_results = model.val(
    data='D:\\Falldown\\yaml_info_yolov8s.yaml',
    imgsz=640,
    batch=32,
    device=device
)

# 평가 종료 시간 기록
end_time = time.time()
execution_time = end_time - start_time

# 결과 출력
print(f"Validation Results:")
print(f"mAP50: {val_results.box.map50:.4f}")
print(f"mAP50-95: {val_results.box.map:.4f}")
print(f"실행 시간: {execution_time:.4f} 초")

Model summary (fused): 168 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning D:\Falldown\Dataset\Resized_Dataset\Val\labels.cache... 9440 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9440/9440 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 295/295 [01:22<00:00,  3.57it/s]


                   all       9440       9440      0.982      0.661      0.827       0.81
              Non_Fall       9440       9440      0.982      0.661      0.827       0.81
Speed: 0.2ms preprocess, 5.4ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\detect\val
Validation Results:
mAP50: 0.8267
mAP50-95: 0.8099
실행 시간: 114.2418 초


In [6]:
# 테스트 시작 시간 기록
start_time = time.time()

# 테스트 실행
test_results = model.val(
    data='D:\\Falldown\\yaml_info_yolov8s.yaml',
    imgsz=640,
    batch=32,
    device=device,
    split="test"  # Test 데이터셋으로 평가
)

# 테스트 종료 시간 기록
end_time = time.time()
execution_time = end_time - start_time

# 테스트 결과 출력
print(f"Test Results:")
print(f"mAP50: {test_results.box.map50:.4f}")
print(f"mAP50-95: {test_results.box.map:.4f}")
print(f"실행 시간: {execution_time:.4f} 초")

val: Scanning D:\Falldown\Dataset\Resized_Dataset\Test\labels... 9580 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9580/9580 [00:04<00:00, 2359.04it/s]


val: New cache created: D:\Falldown\Dataset\Resized_Dataset\Test\labels.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 300/300 [01:24<00:00,  3.56it/s]


                   all       9580       9580      0.985      0.674      0.833      0.815
              Non_Fall       9580       9580      0.985      0.674      0.833      0.815
Speed: 0.2ms preprocess, 5.4ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\detect\val2
Test Results:
mAP50: 0.8332
mAP50-95: 0.8146
실행 시간: 117.0793 초
